# Description

**TODO**

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd

import conf

# Settings

In [3]:
PREDICTION_METHOD="Gene-based"

In [4]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"]
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses')

In [5]:
OUTPUT_DATA_DIR = Path(OUTPUT_DIR, "data")
display(OUTPUT_DATA_DIR)
OUTPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data')

In [6]:
OUTPUT_RAW_DATA_DIR = Path(OUTPUT_DATA_DIR, "raw")
display(OUTPUT_RAW_DATA_DIR)
OUTPUT_RAW_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw')

In [7]:
OUTPUT_PROJ_DATA_DIR = Path(OUTPUT_DATA_DIR, "proj")
display(OUTPUT_PROJ_DATA_DIR)
OUTPUT_PROJ_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/proj')

In [8]:
OUTPUT_PREDICTIONS_DIR = Path(OUTPUT_DIR, "predictions")
display(OUTPUT_PREDICTIONS_DIR)
OUTPUT_PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions')

# Load PharmacotherapyDB gold standard

In [9]:
gold_standard = pd.read_pickle(
    Path(OUTPUT_DIR, "gold_standard.pkl"),
)

In [10]:
gold_standard.shape

(998, 3)

In [11]:
gold_standard.head()

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


In [12]:
doids_in_gold_standard = set(gold_standard['trait'])

# Load LINCS

## Raw data

In [13]:
# TODO: hardcoded
input_file = Path(
    OUTPUT_RAW_DATA_DIR,
    "lincs-data.pkl"
).resolve()

display(input_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/raw/lincs-data.pkl')

In [14]:
lincs_projection = pd.read_pickle(input_file)

In [15]:
lincs_projection.shape

(7120, 1170)

In [16]:
lincs_projection.head()

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
ENSG00000196839,-1.001,-1.835,1.391,1.132,0.257,1.932,0.508,1.408,0.777,0.032,...,-1.692,-0.516,-1.435,-0.317,-0.012,0.641,-0.230,-0.518,-0.177,2.146
ENSG00000170558,1.146,-1.863,0.011,-1.020,1.143,-0.115,1.327,0.310,-1.853,0.872,...,0.354,0.498,0.268,-1.084,-0.142,-0.077,0.633,-1.807,0.032,0.135
ENSG00000117020,-0.693,1.694,-0.804,-0.164,1.145,-1.465,1.221,-0.747,0.829,-0.961,...,-1.196,-0.230,-1.049,-0.347,0.586,0.865,-0.021,2.180,-0.956,0.105
ENSG00000133997,-0.037,0.383,0.269,-0.997,0.185,-0.536,0.424,-0.119,-1.313,0.579,...,-0.343,0.116,-0.245,-0.127,-1.367,0.149,0.117,2.084,1.178,0.772
ENSG00000101473,0.162,-0.899,0.105,-0.090,-1.291,1.404,0.185,0.157,-0.327,-0.026,...,-0.136,-1.115,-0.280,0.200,0.638,-0.197,-0.360,-2.302,-0.117,-0.167


# Load S-PrediXcan

In [17]:
from entity import Trait

In [18]:
phenomexcan_input_file_list = [
    f for f in OUTPUT_RAW_DATA_DIR.glob("*.pkl") if f.name.startswith(('smultixcan-', 'spredixcan-'))
]

In [19]:
display(len(phenomexcan_input_file_list))

51

# Predict drug-disease associations

In [20]:
for phenomexcan_input_file in phenomexcan_input_file_list:
    print(phenomexcan_input_file.name)
    
    # read phenomexcan data
    phenomexcan_projection = pd.read_pickle(phenomexcan_input_file)
    print(f"  shape: {phenomexcan_projection.shape}")
    
    # prediction
    print(f"  predicting...")
    common_genes = phenomexcan_projection.index.intersection(lincs_projection.index)
    phenomexcan_projection = phenomexcan_projection.loc[common_genes]
    lincs_projection = lincs_projection.loc[common_genes]
    drug_disease_assocs = lincs_projection.T.dot(phenomexcan_projection)
    print(f"    shape: {drug_disease_assocs.shape}")
    drug_disease_assocs = Trait.map_to_doid(drug_disease_assocs, doids_in_gold_standard, combine="max")
    print(f"    shape (after DOID map): {drug_disease_assocs.shape}")
    assert drug_disease_assocs.index.is_unique
    assert drug_disease_assocs.columns.is_unique
    
    # build classifier data
    print(f"  building classifier data...")
    classifier_data = drug_disease_assocs\
        .unstack().reset_index()\
        .rename(columns={'level_0': 'trait', 'perturbagen': 'drug', 0: 'score'})
    assert classifier_data.shape == classifier_data.dropna().shape
    print(f"    shape: {classifier_data.shape}")
    display(classifier_data.describe())
    
    # save
    output_file = Path(
        OUTPUT_PREDICTIONS_DIR,
        f"{phenomexcan_input_file.stem}-prediction_scores.h5"
    ).resolve()
    print(f"    saving to: {str(output_file)}")
    
    classifier_data.to_hdf(output_file, mode="w", complevel=4, key='prediction')
    
    pd.Series({
        'method': PREDICTION_METHOD,
        'data': phenomexcan_input_file.stem,
    })\
    .to_hdf(output_file, mode="r+", key='metadata')
    
    print("")

smultixcan-mashr-zscores-data.pkl
  shape: (22152, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 352)
  building classifier data...
    shape: (411840, 3)


,score
count,411840.000000
mean,-3.373597
std,620.107049
min,-6941.775359
25%,-269.519901
50%,19.379591
75%,272.625995
max,27630.148950


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/smultixcan-mashr-zscores-data-prediction_scores.h5

spredixcan-mashr-zscores-Adipose_Subcutaneous-data.pkl
  shape: (14063, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 352)
  building classifier data...
    shape: (411840, 3)


,score
count,411840.000000
mean,20.069349
std,161.777082
min,-6395.271773
25%,-32.454441
50%,16.804851
75%,65.989913
max,6845.743694


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/spredixcan-mashr-zscores-Adipose_Subcutaneous-data-prediction_scores.h5

spredixcan-mashr-zscores-Adipose_Visceral_Omentum-data.pkl
  shape: (13942, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 352)
  building classifier data...
    shape: (411840, 3)


...

    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/spredixcan-mashr-zscores-Vagina-data-prediction_scores.h5

spredixcan-mashr-zscores-Whole_Blood-data.pkl
  shape: (12067, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 352)
  building classifier data...
    shape: (411840, 3)


,score
count,411840.000000
mean,7.961759
std,67.561914
min,-3533.802223
25%,-12.455858
50%,7.287144
75%,27.133235
max,2586.596044


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/spredixcan-mashr-zscores-Whole_Blood-data-prediction_scores.h5

spredixcan-most_signif-data.pkl
  shape: (22152, 4091)
  predicting...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 352)
  building classifier data...
    shape: (411840, 3)


,score
count,411840.000000
mean,14.809420
std,113.128260
min,-4905.335298
25%,-21.055511
50%,12.177767
75%,45.601941
max,6126.926668


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/spredixcan-most_signif-data-prediction_scores.h5

